In [1]:
#Adding the libraries
import tensorflow as tf
import numpy as np
import os
import pandas as pd

In [2]:
#diresctory of the train and test dataset
train_dir = r"train"
test_dir = r"test"
os.path.exists(test_dir)

True

In [3]:
#reading the dataset which have all labels of all images
df = pd.read_csv('Training_set.csv')
df.shape

(6499, 2)

In [4]:
uniq = df['label'].unique()

In [5]:
unique = np.array(df['label'].unique())

In [6]:
import shutil
def move_images(species):
    for images in df.loc[df['label'] == species, 'filename']:
        shutil.move(os.path.join(train_dir, images), os.path.join(os.path.join(train_v2_dir, species), images))

In [7]:
#for labels in uniq:
#    move_images(labels)

In [ ]:
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [8]:
train_v2_dir = r"train_v2"
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=train_v2_dir,
    labels='inferred',
    validation_split=0.2,
    subset='training',
    batch_size=67,
    shuffle=True,
    color_mode='rgb',
    seed=123,
    image_size = (128,128)
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=train_v2_dir,
    labels='inferred',
    validation_split=0.2,
    subset='validation',
    batch_size=67,
    shuffle=True,
    color_mode='rgb',
    seed=123,
    image_size = (128,128)
)
test_ds = tf.keras.utils.image_dataset_from_directory(
    directory=test_dir,
    labels=None,
    batch_size=67,
    shuffle=True,
    color_mode='rgb',
    image_size = (128,128)
)

Found 6499 files belonging to 75 classes.
Using 5200 files for training.
Found 6499 files belonging to 75 classes.
Using 1299 files for validation.
Found 2786 files.


In [9]:
train_ds_final = (train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE))
val_ds_final = (val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE))
test_ds_final = (test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE))

In [10]:
aug = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomTranslation(0.2,0.2)
])
rescale_input = tf.keras.Sequential([
    tf.keras.layers.Resizing(100,100),
    tf.keras.layers.Rescaling(1./255)
])

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(shape=(128,128,3)),
    rescale_input,
    aug,
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(75,activation='softmax')
])

In [12]:
adam = tf.keras.optimizers.Adam(learning_rate=0.001,epsilon=1e-07)

In [13]:
model.compile(loss= tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=adam, metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_1 (Sequential)       │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 100, 100, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 50, 50, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 48, 48, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 22, 22, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 11, 11, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 75)             │         9,675 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 349,707 (1.33 MB)

 Trainable params: 349,707 (1.33 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(train_ds_final, validation_data=val_ds_final, epochs=200)